In [108]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from sklearn.cross_validation import train_test_split
from sklearn import preprocessing
import dateutil.parser
#yourdate = dateutil.parser.parse(datestring)
from time import gmtime, strftime

In [109]:
needed = ["NAME", "MON/WED/FRI (FRIDAYS UNTIL 5:00 PM)", "TUE/THU", "SATURDAY", "SUNDAY"]

In [110]:
#ref is column entry, col is column name
def check(ref, col):
    day = [""]
    time = ""
    #MWF
    if col == needed[1]:
        day = [days[0], days[2], days[4]]
    #TT
    elif col == needed[2]:
        day = [days[1], days[3]]
    #SAT
    elif col == needed[3]:
        day = [days[5]]
    #SUN
    elif col == needed[4]:
        day = [days[6]]

    for i in range(0, len(day)):
        #Check if time is empty to avoid comma at the start and build the string
        if not time:
            time = day[i] + ", " +duration(day[i], ref)
        else:
            time = time +"; "+ day[i] + "," +duration(day[i], ref)
    return time

    #return day

In [111]:
def duration(day, reference):
    starttime = stoptime = 'null' 
    #print("Entered duration")
    if day.lower() == "Sunday".lower() or day.lower() == "Satuday".lower():
            starttime = stoptime = dateutil.parser.parse("12:00AM").time()  
            
            if reference.find("Afternoon") != -1:
                starttime = dateutil.parser.parse("1PM").time()
                if day.lower() == "Sunday".lower():
                    stoptime = dateutil.parser.parse("6PM") .time() 
                else:
                    stoptime = dateutil.parser.parse("5PM") .time()
                    
            elif reference.find("Closing") != -1:
                starttime = dateutil.parser.parse("6PM").time()
                stoptime = dateutil.parser.parse("10PM") .time() 
                
    else:
        starttime = stoptime = dateutil.parser.parse("12:00AM").time()
        #print("Not Sunday or Saturday")
        if reference.find("Opening") != -1:
            #print("Entered Opening")
            starttime = dateutil.parser.parse("7:30AM").time()
            stoptime = dateutil.parser.parse("10AM") .time()   
             
        elif reference.find("Morning") != -1:
            #print("Entered Morning")  
            starttime = dateutil.parser.parse("10AM").time()
            stoptime = dateutil.parser.parse("1PM") .time() 
         
        elif reference.find("Mid Day") != -1:
            #print("Entered Mid Day")  
            starttime = dateutil.parser.parse("1PM").time()
            if day.lower() == "Friday".lower():
                stoptime = dateutil.parser.parse("5PM") .time() 
            else:
                stoptime = dateutil.parser.parse("4PM") .time() 
                
        elif reference.find("Afternoon") != -1:
            starttime = dateutil.parser.parse("4PM").time()
            stoptime = dateutil.parser.parse("7PM") .time() 
            
        elif reference.find("Closing") != -1:
            starttime = dateutil.parser.parse("7PM").time()
            stoptime = dateutil.parser.parse("10PM") .time() 
                
    timeframe = starttime.strftime("%I:%M %p") + " - " + stoptime.strftime("%I:%M %p")            
    return timeframe

In [112]:
duration('Friday', 'Afternoon')

'04:00 PM - 07:00 PM'

In [113]:
#Testing Check function
check('Mid Day (1pm-4pm) (Fridays 1pm-5pm)','MON/WED/FRI (FRIDAYS UNTIL 5:00 PM)')

'Monday, 01:00 PM - 04:00 PM; Wednesday,01:00 PM - 04:00 PM; Friday,01:00 PM - 05:00 PM'

In [114]:
check('Morning (10am-1pm)', 'TUE/THU')

'Tuesday, 10:00 AM - 01:00 PM; Thursday,10:00 AM - 01:00 PM'

In [115]:
data = pd.read_csv("/Users/Rohit/Desktop/schedule.csv")
data.columns = map(str.upper, data.columns)

# Data Management
#list(data.columns.values)

In [116]:
clean_data = data[needed] #Necessary Columns
#clean_data

In [117]:
# Table (Name, Availability) columns for each section
for i in range(1, clean_data.shape[1]):
    temp = clean_data[pd.notnull(clean_data[needed[i]])]
    
    #Needed data at a time
    col = [needed[0], needed[i]]
    temp = temp[col] 
    
    #Splitting shifts initially from csv dataframe import
    s = temp[needed[i]].str.split(',').apply(Series, 1).stack()
    s.index = s.index.droplevel(-1)
    s.name = needed[i]
    del temp[needed[i]]
    temp = temp.join(s).reset_index(drop=True)
    
    #Applying check() to convert to readable form for every row (shape[0])
    for j in range(0, temp.shape[0]): 
        #Cleaning the 2nd column
        temp.at[j, needed[i]] =  check(temp.at[j, needed[i]], needed[i]) #check(reference, day)
        #Splitting single availability column to day-by-day columns
        new = pd.DataFrame(temp[needed[i]].apply(lambda x: x.split('; ')).values.tolist())
        #Merge the day-by-day with the name column to get the desired result
        result = pd.concat([temp[needed[0]], new], axis=1)
        
        
    #Test Case
    if(i == 1):
        mwf = result
    elif(i == 2):
        tt = result
    elif(i == 3):
        sat = result
    elif(i == 4):
        sun = result

#mwf

In [118]:
#Monday
mon = mwf[[needed[0], 0]]
#Wednesday
wed = mwf[[needed[0], 1]]
#Friday
fri = mwf[[needed[0], 2]]
#Tuesday
tue = tt[[needed[0], 0]]
#Thursday
thu = tt[[needed[0], 1]]

week = [mon, tue, wed, thu, fri, sat, sun]
counter = 0 
weekname = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
weekname[0]

'Monday'

In [119]:
def daysplit(ref):
    return ref.split(",")[1]

def starttime(ref):
    return dateutil.parser.parse(ref.split("-")[0]).time()

def stoptime(ref):
    return dateutil.parser.parse(ref.split("-")[1]).time()

In [120]:
starttime(daysplit("Monday, 07:30 AM - 10:00 AM"))

datetime.time(7, 30)

In [121]:
stoptime(daysplit("Monday, 01:00 PM - 04:00 PM"))

datetime.time(16, 0)

In [122]:

#outfile 
for day in week: 
    for i in range(0, len(day)):
        #day[day.columns[1]].str.replace(day[day.columns[1]], day[day.columns[1]].str.split(',')[i][1])
        #WORKS! 
        #day[day.columns[1]].apply(lambda x: finalsplitter(x))
        
        day.loc[i,day.columns[1]] = daysplit(day.loc[i,day.columns[1]])
        day["START"] = day[day.columns[1]].apply(lambda x: starttime(x))
        day["STOP"] = day[day.columns[1]].apply(lambda x: stoptime(x))
    new = day.sort_values(by="START")
    day = new
    day.to_csv(r"/Users/Rohit/Desktop/%s.txt" % str(weekname[counter]), header='\n', index=None, mode='a', sep='\t')
    day.to_html(r"/Users/Rohit/Desktop/%s.txt" % str(weekname[counter]))
    counter = counter + 1
    print(day)
    
    
f.close()
#

//anaconda/lib/python3.5/site-packages/pandas/core/indexing.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


                     NAME                     0     START      STOP
0             Davis Allen   07:30 AM - 10:00 AM  07:30:00  10:00:00
1   Dante Dalla Gasperina   07:30 AM - 10:00 AM  07:30:00  10:00:00
11         Jessica Battad   07:30 AM - 10:00 AM  07:30:00  10:00:00
27           Parker Gribb   07:30 AM - 10:00 AM  07:30:00  10:00:00
17         Andrew Schmidt   10:00 AM - 01:00 PM  10:00:00  13:00:00
14           Pranav Bheda   10:00 AM - 01:00 PM  10:00:00  13:00:00
12         Jessica Battad   10:00 AM - 01:00 PM  10:00:00  13:00:00
23     Sanjay Tamizharasu   10:00 AM - 01:00 PM  10:00:00  13:00:00
21          Robert Scalzo   10:00 AM - 01:00 PM  10:00:00  13:00:00
5          Grecia Escobar   10:00 AM - 01:00 PM  10:00:00  13:00:00
3                 JT Tran   10:00 AM - 01:00 PM  10:00:00  13:00:00
9       Francesco Petrini   01:00 PM - 04:00 PM  13:00:00  16:00:00
6          Grecia Escobar   01:00 PM - 04:00 PM  13:00:00  16:00:00
4                 JT Tran   01:00 PM - 04:00 PM 